This file shows the steps taken when generating the Namarsh dataset

The first step was to extract data from .html files

The data is: the name of the sections from the website and texts from the articles

In [1]:
# The library to extract data from HTML
from bs4 import BeautifulSoup
# The library used for data manipulation: https://pandas.pydata.org
import pandas as pd
# The libraries to access local files and folders which will be used to iterate over thousands of .html files in the namarsh folder
import os
import glob

Further, the folder that contains Namarsh HTML files in defined.

full = [] creates a new array where processed data will then be stored during the next step.

In [2]:
files = glob.glob('materials/*.html')
full = []

The next step is to iterate over the HTML files in the folder and pull the website articles.

The records are then further stored in the dataframe.

The subsections are found by <p></p> HTML tag which contains the articles.

The search is done using the BeautifulSoup library.

All the data is then converted to a data frame for further manipulation.

In [3]:
for file in files:

    extractor = open(file, 'rb')
    soup = BeautifulSoup(extractor, features="html.parser")

    text = soup.find_all("div", class_= "content-container")
    p_text = [p.get_text() for p in text]

    full.append([p_text, os.path.basename(file)])

In [8]:
df_full = pd.DataFrame(full)
df_full.head()

,0,1
0,[\nНовости протеста\n\t15.05.2012 Время извест...,4FB28BA0CF235.html
1,[\nНовости протеста\n\n\t24.08.2018 Власти Мос...,5B800AF319A17.html
2,[\nАкции\n\n\t01.01.1970 \n\n\n\n\n\n],59EA075BB1F0C.html
3,[\nГлазами очевидца\n\n\t21.04.2009 Слепая вла...,49ED883B170D0.html
4,[\nАкции\n\n\t24.09.2015 В Новосибирске отметя...,5603E68F0791B.html


Now we are creating a function that will run over the dataframe columns and find News of Protest subsection.

The first column of df_full is being renamed into ['Name']

In [9]:
df_full = df_full[df_full[0].astype('str').str.contains("Новости протеста")]

In [10]:
df_full.head()

,0,1
0,[\nНовости протеста\n\t15.05.2012 Время извест...,4FB28BA0CF235.html
1,[\nНовости протеста\n\n\t24.08.2018 Власти Мос...,5B800AF319A17.html
7,[\nНовости протеста\n\n\t24.05.2013 Возле МГУ ...,519F676C06656.html
8,[\nНовости протеста\n\n\t10.05.2013 Двое из за...,518C988E2768B.html
11,[\nНовости протеста\n\n\t04.04.2008 На свет бо...,47F5F8605688C.html


Create a new dataset that will conveniently store cleaner data for the dataset

Then the function is applied and null values are removed using dropna() pandas function that does this

The output of the function is then stored in the clean dataframe

In [14]:
clean_df = pd.DataFrame()
clean_df['Name'] = df_full[0].dropna()

In [18]:
clean_df['Name']

0       [\nНовости протеста\n\t15.05.2012 Время извест...
1       [\nНовости протеста\n\n\t24.08.2018 Власти Мос...
7       [\nНовости протеста\n\n\t24.05.2013 Возле МГУ ...
8       [\nНовости протеста\n\n\t10.05.2013 Двое из за...
11      [\nНовости протеста\n\n\t04.04.2008 На свет бо...
                              ...                        
6634    [\nНовости протеста\n\n\t16.12.2015 Скорая пом...
6636    [\nНовости протеста\n\n\t24.03.2018 В Новосиби...
6638    [\nНовости протеста\n\t07.02.2008 Выборы все б...
6639    [\nНовости протеста\n\n\t16.01.2017 Петербург ...
6641    [\nНовости протеста\n\n\t19.07.2018 Власть уст...
Name: Name, Length: 3235, dtype: object

Since data in the <p></p> still included more markup, the next steps are to get rid of it and split the data into dataset columns.

The columns are: title of the event, description, short description (which will be used as the event identifier or title when merged with the Forthcoming Events subsection).

The data is stripped in the order it is presented in the articles (title short description in the first paragraph followed by the main description in the second and on).

The split is done using the \t and \n symbols which were pulled during the <p></p> step together with the data.

The \n symbols are "new line", and new lines are used to begin new sections.

The 'output_df' will store the final version of the News of Protest subsection

In [33]:
#The date column because <p></p> started with the date
output_df = clean_df['Name'].astype('str').str.split("t", n=1, expand=True)
output_df = output_df.drop([0], axis=1)
output_df.head()

,1
0,15.05.2012 Время известно\nПо решению суда лаг...
1,24.08.2018 Власти Москвы отказались рассматрив...
7,24.05.2013 Возле МГУ прошел пикет в защиту сту...
8,10.05.2013 Двое из задержанных в гайд-парке в ...
11,"04.04.2008 На свет божий \nШахтеры ""Красной ша..."


In [35]:
output_df = output_df[1].astype('str').str.split(" ", n=1, expand=True)
output_df.head()

,0,1
0,15.05.2012,Время известно\nПо решению суда лагерь на Чист...
1,24.08.2018,Власти Москвы отказались рассматривать заявку ...
7,24.05.2013,"Возле МГУ прошел пикет в защиту студентов — ""у..."
8,10.05.2013,Двое из задержанных в гайд-парке в Москве все ...
11,04.04.2008,"На свет божий \nШахтеры ""Красной шапочки"" в Се..."


In [36]:
#The title and description columns. The latter contains both the introduction paragraph and main description
output_df[['title', 'description']] = output_df[1].astype('str').str.split("n", n=1, expand=True)
output_df['title'] = output_df['title'].astype('str').str.strip("\\")
output_df['description'] = output_df['description'].astype('str').str.strip(r"\n")
output_df = output_df.drop([1], axis=1)

#Split description in "short description" and "details"
output_df[['short_description', 'details']] = output_df['description'].astype('str').str.split("n", n=1, expand=True)
output_df['short_description'] = output_df.short_description.astype('str').str.strip("\\")
output_df['details'] = output_df.details.astype('str').str.strip(r"\n")

# Get rid of the original column since it has been split
output_df = output_df.drop(['description'], axis=1)
output_df.head()

,0,title,short_description,details
0,15.05.2012,Время известно,По решению суда лагерь на Чистых прудах будет ...,На Чистопрудном бульваре было зачитано постан...
1,24.08.2018,Власти Москвы отказались рассматривать заявку ...,Московская мэрия отказалась рассматривать зая...,"В мэрии заявили, что заявка подана раньше срок..."
7,24.05.2013,"Возле МГУ прошел пикет в защиту студентов — ""у...",Возле здания Московского государственного уни...,Организаторами пикета выступили инициативные г...
8,10.05.2013,Двое из задержанных в гайд-парке в Москве все ...,В Москве гражданские активисты провели днем 9...,"По инфоормации Яшина,\n\nв гайд-парк пришло ок..."
11,04.04.2008,На свет божий,"Шахтеры ""Красной шапочки"" в Североуральске доб...","Рабочие шахты ""Красная шапочка"" в Североураль..."


Now that the data is split into columns, the next step is to remove unnecessary symbols that contaminate the dataset.

For this, the cleaner() function is introduced which replaces the symbols with " "

In [37]:
def cleaner(data):
    lister = {r'\r', r'\n', r'\xa0'}
    for i in lister:
        data = data.replace(i, " ")
    return data

The function is then applied to relevant columns with extra symbols

In [39]:
output_df['short_description'] = output_df['short_description'].apply(str).apply(lambda x: cleaner(x))
output_df['details'] = output_df['details'].apply(str).apply(lambda x: cleaner(x))

# Additional cleaning
output_df['short_description'] = output_df['short_description'].astype('str').str.replace(r'\s+', ' ', regex=True)
output_df['details'] = output_df['details'].astype('str').str.replace(r'\s+', ' ', regex=True)
output_df['details'] = output_df['details'].astype('str').str.strip(r'\']')

output_df.head()

,0,title,short_description,details
0,15.05.2012,Время известно,По решению суда лагерь на Чистых прудах будет ...,На Чистопрудном бульваре было зачитано постан...
1,24.08.2018,Власти Москвы отказались рассматривать заявку ...,Московская мэрия отказалась рассматривать зая...,"В мэрии заявили, что заявка подана раньше срок..."
7,24.05.2013,"Возле МГУ прошел пикет в защиту студентов — ""у...",Возле здания Московского государственного уни...,Организаторами пикета выступили инициативные г...
8,10.05.2013,Двое из задержанных в гайд-парке в Москве все ...,В Москве гражданские активисты провели днем 9...,"По инфоормации Яшина, в гайд-парк пришло около..."
11,04.04.2008,На свет божий,"Шахтеры ""Красной шапочки"" в Североуральске доб...","Рабочие шахты ""Красная шапочка"" в Североураль..."


To extract locations out of the data, string matching needs to be done.

For that, a list of Russian cities is exported and iterated one by one.

To find out if the description contains any of the cities from the list, it needs to be iterated over.

The matching lines are then stored in an array.

The array is then connected to the dataframe.

This is done using the city_extractor() function.

In [48]:
# The list of cities is imported
cities_list = pd.read_csv('towns.csv')

def city_extractor(row):
    for cities in cities_list['city']:
        if cities in row.short_description: return cities
        #if row.cities == 'None' and cities in row.details: return cities
    return 'None'

#This line is specific to the Russian language because it has the declension system which changes the ending of words
#The purpose is to remove the vowels from the ending to keep the root of the word and use this root to find matches
cities_list['city'] = cities_list['city'].apply(lambda x: x[:-1] if x[-1] in set('аея') else x)

The function is then applied

The words with missing ending vowels are returned

In [50]:
output_df['cities'] = (output_df.apply(lambda x: city_extractor(x), axis=1))

replacements = {
    'cities': {
        "Москв" : "Москва", "Калуг" : "Калуга", "Тул" : "Тула", "Самар" : "Самара", "Раменск" : "Раменское",
        "Балаших" : "Балашиха", "Костром" : "Кострома", "Вологд" : "Вологда", "Кашир" : "Кашира", "Донецк" : "None",
        "Козловк" : "Козловка", "Махачкал" : "Махачкала", "Туапс" : "Туапсе",
        "Чит" : "Чита", "Пенз" : "Пенза", "Каменк" : "Каменка", "Кушв" : "Кушва", "Тарус" : "Таруса",
        "Ухт" : "Ухта", "Уф" : "Уфа", "Находк" : "Находка", "Балахн" : "Балахна", "Юрг" : "Юрга", "Раменско" : "Раменское",
        "Лобн" : "Лобна"}
    }

output_df.replace(replacements, inplace=True)
output_df.head()

,0,title,short_description,details,cities
0,15.05.2012,Время известно,По решению суда лагерь на Чистых прудах будет ...,На Чистопрудном бульваре было зачитано постан...,None
1,24.08.2018,Власти Москвы отказались рассматривать заявку ...,Московская мэрия отказалась рассматривать зая...,"В мэрии заявили, что заявка подана раньше срок...",Инт
7,24.05.2013,"Возле МГУ прошел пикет в защиту студентов — ""у...",Возле здания Московского государственного уни...,Организаторами пикета выступили инициативные г...,Москва
8,10.05.2013,Двое из задержанных в гайд-парке в Москве все ...,В Москве гражданские активисты провели днем 9...,"По инфоормации Яшина, в гайд-парк пришло около...",Москва
11,04.04.2008,На свет божий,"Шахтеры ""Красной шапочки"" в Североуральске доб...","Рабочие шахты ""Красная шапочка"" в Североураль...",Североуральск


The alternative way is to use the Pymorhy2 library that applies declension to words.

It would be more precise, but will take more time and resources if applied on a larger dataset

Here is the way:

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def match_city(row):
    for city in cities_list['city']:
        morpheme = morph.parse(city)[0]
        # Make sure that the city is found as a separate word
        city_word = city + " "
        genitiv = morpheme.inflect({'gent'})[0] + " "
        dativ = morpheme.inflect({'dativ'})[0] + " "
        locativ = morpheme.inflect({'loct'})[0] + " "
        ablativ = morpheme.inflect({'ablt'})[0] + " "
        if (city_word.lower() in row['text'].lower()) or (ablativ.lower() in row['text'].lower()) or (genitiv.lower() in row['text'].lower()) or (dativ.lower() in row['text'].lower()) or (locativ.lower() in row['text'].lower()):
            return city
    return 'None'

output_df['cities'] = csv.apply(match_city, axis=1)

The last step is to save the output to CSV file

In [51]:
output_df.to_csv("news-of-protest.csv", index=False)